In [ ]:
# etl by semio

In [67]:
import os
import numpy as np
import pandas as pd
from ddf_utils.str import format_float_digits

source_dir = '../source'
bracket_file = os.path.join(source_dir, 'brackets.csv')


In [104]:
from functools import partial

In [106]:
formattor = partial(format_float_digits, digits=6)

In [2]:
def load_file_preprocess(filename, income_bracket):
    usecols = ['CountryCode','CountryName','CoverageType','RequestYear', 'PovertyLine', 'HeadCount','ReqYearPopulation']
    df = pd.read_csv(os.path.join(source_dir, filename), usecols=usecols)
    df = df.rename(columns={'PovertyLine': 'income_bracket'})
    df['income_bracket'] = income_bracket
    df = df.set_index(['CountryCode', 'RequestYear', 'CoverageType', 'income_bracket'])
    return df

In [3]:

all_source_files = os.listdir(source_dir)
all_source_files = [x for x in all_source_files if x.startswith('0')]
all_source_files.sort()
brackets = pd.read_csv(bracket_file)
# use brackets end to indentity income_brackets
brackets_entity_mapping = brackets['bracket_end'].to_dict()
brackets_entity_mapping_reverse = dict([(v, k) for k, v in brackets_entity_mapping.items()])

source_pairs = list(zip(all_source_files[:-1], all_source_files[1:]))  # a list of file name pairs (bracket start, bracket end)


In [107]:
population_list = []
population_percentage_list = []

country_list = []

for i, files in enumerate(source_pairs):
    start_file, end_file = files
    df_start = load_file_preprocess(start_file, i)
    df_end = load_file_preprocess(end_file, i)
    
    # collect geos
    geo1 = df_start['CountryName'].reset_index([1, 2, 3], drop=True).drop_duplicates()
    geo2 = df_end['CountryName'].reset_index([1, 2, 3], drop=True).drop_duplicates()
    country_list.append(geo1)
    country_list.append(geo2)
    
    # collect datapoints
    population_percentage = df_end['HeadCount'] - df_start['HeadCount']
    
    population_start = df_start['ReqYearPopulation'] * df_start['HeadCount'] * 1000000
    population_end = df_end['ReqYearPopulation'] * df_end['HeadCount'] * 1000000
    population = population_end - population_start
    
    # population = population_percentage.multiply(df_start['ReqYearPopulation'], axis=0)
    
    population_list.append(population)
    population_percentage_list.append(population_percentage)
# concat
df_population = pd.concat(population_list)
df_population_percentage = pd.concat(population_percentage_list)

In [108]:
df_population

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                  0.000000
DZA          1981         N             0                  0.000000
AGO          1981         N             0                  0.000000
ARG          1981         U             0                 16.149012
ARM          1981         N             0                  1.750971
                                                            ...    
VNM          2019         N             99                 0.000000
PSE          2019         N             99                 0.000000
YEM          2019         N             99                 0.000000
ZMB          2019         N             99                 0.000000
ZWE          2019         N             99                 0.000000
Length: 664500, dtype: float64

In [109]:
df_population = df_population.reset_index()

In [110]:
df_population.columns = ['geo', 'year', 'coverage_type', 'income_bracket', 'population']
df_population['geo'] = df_population['geo'].str.lower()
df_population['coverage_type'] = df_population['coverage_type'].str.lower()

# TODO: notify if any
df_population = df_population[df_population['population'] >= 0]

In [111]:
df_population['population'] = df_population['population'].map(formattor)

In [112]:
df_population = df_population.sort_values(by=['geo', 'year', 'coverage_type', 'income_bracket'])

In [113]:
df_population.to_csv('../../ddf--datapoints--population--by--geo--year--coverage_type--income_bracket.csv', index=False)

In [114]:
df_population_percentage = df_population_percentage.reset_index()

In [115]:
df_population_percentage.columns = ['geo', 'year', 'coverage_type', 'income_bracket', 'population_percentage']
df_population_percentage['geo'] = df_population_percentage['geo'].str.lower()
df_population_percentage['coverage_type'] = df_population_percentage['coverage_type'].str.lower()

# TODO: notify if any
df_population_percentage = df_population_percentage[df_population_percentage['population_percentage'] >= 0]

In [116]:
df_population_percentage['population_percentage'] = df_population_percentage['population_percentage'].map(formattor)

In [117]:
df_population_percentage = df_population_percentage.sort_values(by=['geo', 'year', 'coverage_type', 'income_bracket'])

In [118]:
df_population_percentage.to_csv('../../ddf--datapoints--population_percentage--by--geo--year--coverage_type--income_bracket.csv', index=False)

In [65]:
df_population_percentage

,geo,year,coverage_type,income_bracket,population
0,alb,1981,n,0,0.000000e+00
1,dza,1981,n,0,0.000000e+00
2,ago,1981,n,0,0.000000e+00
3,arg,1981,u,0,6.840000e-07
4,arm,1981,n,0,5.562000e-07
...,...,...,...,...,...
664495,vnm,2019,n,99,0.000000e+00
664496,pse,2019,n,99,0.000000e+00
664497,yem,2019,n,99,0.000000e+00
664498,zmb,2019,n,99,0.000000e+00


In [ ]:
# entities

In [35]:
coverage_type_entities = pd.DataFrame(data = {'coverage_type':['n','u','a','r'], 'name':['National','Urban','Aggregated','Rural']})

coverage_type_entities

,coverage_type,name
0,n,National
1,u,Urban
2,a,Aggregated
3,r,Rural


In [36]:
coverage_type_entities.to_csv("../../ddf--entities--coverage_type.csv", index=False)

In [41]:
df_brackets = brackets.copy()
df_brackets['name'] = [ str(x) + ' - ' + str(y) for x, y in zip(df_brackets['bracket_start'], df_brackets['bracket_end'])]
df_brackets

,bracket_start,bracket_end,name
0,0.00781,0.00897,0.00781 - 0.00897
1,0.00897,0.01031,0.00897 - 0.01031
2,0.01031,0.01184,0.01031 - 0.01184
3,0.01184,0.01360,0.01184 - 0.0136
4,0.01360,0.01563,0.0136 - 0.01563
...,...,...,...
95,4096.00000,4705.06846,4096.0 - 4705.06846
96,4705.06846,5404.70440,4705.06846 - 5404.7044
97,5404.70440,6208.37506,5404.7044 - 6208.37506
98,6208.37506,7131.55022,6208.37506 - 7131.55022


In [66]:
df_brackets = df_brackets['name'].reset_index()
df_brackets.columns = ['income_bracket', 'name']

df_brackets.to_csv('../../ddf--entities--income_bracket.csv', index=False)

In [49]:
country_list[0]

CountryCode
ALB        Albania
DZA        Algeria
AGO         Angola
ARG      Argentina
ARM        Armenia
          ...     
XKX         Kosovo
TLS    Timor-Leste
NRU          Nauru
SSD    South Sudan
SOM        Somalia
Name: CountryName, Length: 168, dtype: object

In [53]:
country = pd.concat(country_list).drop_duplicates()

In [54]:
country = country.reset_index()
country.columns = ['geo', 'name']

In [84]:
country['geo'] = country['geo'].str.lower()

In [85]:
country.to_csv('../../ddf--entities--geo.csv', index=False)

In [56]:
disc = pd.DataFrame([
    ['name', 'Name', 'string'],
    ['year', 'Year', 'time'],
    ['geo', 'Geo', 'entity_domain'],
    ['coverage_type','CoverageType', 'entity_domain'],
    ['income_bracket','IncomeBracket', 'entity_domain'],
    ['population', 'Population', 'measure'],
    ['population_percentage', 'Population Percentage', 'measure']
], columns=['concept', 'name', 'concept_type'])


In [57]:
disc

,concept,name,concept_type
0,name,Name,string
1,year,Year,time
2,geo,Geo,entity_domain
3,coverage_type,CoverageType,entity_domain
4,income_bracket,IncomeBracket,entity_domain
5,population,Population,measure
6,population_percentage,Population Percentage,measure


In [58]:
disc.to_csv("../../ddf--concepts.csv", index=False)

In [22]:
i = 0
start_file, end_file = source_pairs[i]

df_start = load_file_preprocess(start_file, i)
df_end = load_file_preprocess(end_file, i)


population_percentage = df_end['HeadCount'] - df_start['HeadCount']

population_start = df_start['ReqYearPopulation'] * df_start['HeadCount'] * 1000000
population_end = df_end['ReqYearPopulation'] * df_end['HeadCount'] * 1000000

population = population_end - population_start
# population = population_percentage.multiply(df_start['ReqYearPopulation'], axis=0) * 1000000

In [23]:
population_percentage

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                 0.000000e+00
DZA          1981         N             0                 0.000000e+00
AGO          1981         N             0                 0.000000e+00
ARG          1981         U             0                 6.840000e-07
ARM          1981         N             0                 5.562000e-07
                                                              ...     
VNM          2019         N             0                 0.000000e+00
PSE          2019         N             0                 0.000000e+00
YEM          2019         N             0                 0.000000e+00
ZMB          2019         N             0                 0.000000e+00
ZWE          2019         N             0                 0.000000e+00
Name: HeadCount, Length: 6645, dtype: float64

In [24]:
population

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                  0.000000
DZA          1981         N             0                  0.000000
AGO          1981         N             0                  0.000000
ARG          1981         U             0                 16.149012
ARM          1981         N             0                  1.750971
                                                            ...    
VNM          2019         N             0                  0.000000
PSE          2019         N             0                  0.000000
YEM          2019         N             0                  0.000000
ZMB          2019         N             0                  0.000000
ZWE          2019         N             0                  0.000000
Length: 6645, dtype: float64

In [7]:
df_start

,,,,CountryName,HeadCount,ReqYearPopulation
CountryCode,RequestYear,CoverageType,income_bracket,,,
ALB,1981,N,0,Albania,0.000000,2.726056
DZA,1981,N,0,Algeria,0.000000,19.824301
AGO,1981,N,0,Angola,0.000000,8.640446
ARG,1981,U,0,Argentina,0.002289,23.609667
ARM,1981,N,0,Armenia,0.000130,3.148096
...,...,...,...,...,...,...
VNM,2019,N,0,Vietnam,0.000000,96.462106
PSE,2019,N,0,West Bank and Gaza,0.000000,4.685306
YEM,2019,N,0,"Yemen, Republic of",0.000000,29.161922


In [16]:
df_end

,,,,CountryName,HeadCount,ReqYearPopulation
CountryCode,RequestYear,CoverageType,income_bracket,,,
ALB,1981,N,0,Albania,0.000000,2.726056
DZA,1981,N,0,Algeria,0.000000,19.824301
AGO,1981,N,0,Angola,0.000000,8.640446
ARG,1981,U,0,Argentina,0.002290,23.609667
ARM,1981,N,0,Armenia,0.000131,3.148096
...,...,...,...,...,...,...
VNM,2019,N,0,Vietnam,0.000000,96.462106
PSE,2019,N,0,West Bank and Gaza,0.000000,4.685306
YEM,2019,N,0,"Yemen, Republic of",0.000000,29.161922


In [17]:
start_file

'0000.csv'

In [18]:
df_start['ReqYearPopulation'] * df_start['HeadCount']

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                 0.000000
DZA          1981         N             0                 0.000000
AGO          1981         N             0                 0.000000
ARG          1981         U             0                 0.054041
ARM          1981         N             0                 0.000411
                                                            ...   
VNM          2019         N             0                 0.000000
PSE          2019         N             0                 0.000000
YEM          2019         N             0                 0.000000
ZMB          2019         N             0                 0.000000
ZWE          2019         N             0                 0.000000
Length: 6645, dtype: float64

In [19]:
df_end['ReqYearPopulation'] * df_end['HeadCount']

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                 0.000000
DZA          1981         N             0                 0.000000
AGO          1981         N             0                 0.000000
ARG          1981         U             0                 0.054057
ARM          1981         N             0                 0.000412
                                                            ...   
VNM          2019         N             0                 0.000000
PSE          2019         N             0                 0.000000
YEM          2019         N             0                 0.000000
ZMB          2019         N             0                 0.000000
ZWE          2019         N             0                 0.000000
Length: 6645, dtype: float64

In [20]:
population_start

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                     0.000000
DZA          1981         N             0                     0.000000
AGO          1981         N             0                     0.000000
ARG          1981         U             0                 54040.638990
ARM          1981         N             0                   410.742789
                                                              ...     
VNM          2019         N             0                     0.000000
PSE          2019         N             0                     0.000000
YEM          2019         N             0                     0.000000
ZMB          2019         N             0                     0.000000
ZWE          2019         N             0                     0.000000
Length: 6645, dtype: float64

In [21]:
population_end

CountryCode  RequestYear  CoverageType  income_bracket
ALB          1981         N             0                    0.000000
DZA          1981         N             0                    0.000000
AGO          1981         N             0                    0.000000
ARG          1981         U             0                 5405.678800
ARM          1981         N             0                   41.249376
                                                             ...     
VNM          2019         N             0                    0.000000
PSE          2019         N             0                    0.000000
YEM          2019         N             0                    0.000000
ZMB          2019         N             0                    0.000000
ZWE          2019         N             0                    0.000000
Length: 6645, dtype: float64